In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in files:
    with open(f) as fopen:
        urls.extend(json.load(fopen))
urls = sorted(list(set(urls)))
len(urls)

24542

In [3]:
!mkdir page

In [4]:
cookies = {
    '_gid': 'GA1.2.1774589498.1688827808',
    '_pcid': '%7B%22browserId%22%3A%22lju4h0iqcndtfxp8%22%7D',
    'cX_P': 'lju4h0iqcndtfxp8',
    '_pctx': '%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAEzIEYOAWAZgAYATAHZ%2BADl68xwwRwEBWQSAC%2BQA',
    '__stgeo': 'IjAi',
    '__stbpnenable': 'MQ==',
    '__stdf': 'MA==',
    'cX_G': 'cx%3A1i0t3fg74zq623k3k9afr0x7u3%3A2nq99z7wrkb6x',
    '__stp': 'eyJ2aXNpdCI6InJldHVybmluZyIsInV1aWQiOiIwYzljYmIyZC02MTliLTQxMGItYTQzZS01YTg1N2EwMjhmODAifQ==',
    'XSRF-TOKEN': 'eyJpdiI6ImZGQlwvVGVHSkc4UE9CMjk1ZGlpcEpBPT0iLCJ2YWx1ZSI6InhrbUlrM2FKZlpROWU5XC91bnFESnJ3WTV1NzBidDdHNXFtaExFUHBNOThXQVBmTVZUTXFTTmVJTzB3bHRra0crNnZOMEIzYUpUcHQ2Tk5ZNVpjNERPdz09IiwibWFjIjoiYzBlOGQ4ZGYxMjIzMzRhZjZmYjg3NjcxYTVjNDJmZDg0OGE1ZjljZWU5NmRkZmFlYTI4MmI3YTZmOTQ5NWQ1MiJ9',
    'carsifu_session': 'eyJpdiI6IkZjUG1DYnQrTkcrS0k0c3ZkUjNxYUE9PSIsInZhbHVlIjoidmgzUm9xaGpqSFJQVkswb2J3SmtyQ3V4emlXbEI5REhuUlJ5R1drRmc3ZDk3UHQwOWVlNzB1TzhyVTYwZmJSamFIWGI2eWU3MGl3c3NCWlI0VnhYeHc9PSIsIm1hYyI6IjIzYWJmM2IyOTM4YzVjZjk4MmU1NzZjZGU5NTgxMjE0YzM4YmQwOGRjYmFiMDcyOTIzNDk4MDBhOTI5ZmM5NTIifQ%3D%3D',
    '_ga_QF13J11YBW': 'GS1.1.1688889128.2.1.1688889384.35.0.0',
    '_ga': 'GA1.1.2139008171.1688827808',
    '__sts': 'eyJzaWQiOjE2ODg4ODkxMjgzODEsInR4IjoxNjg4ODg5Mzg0MjgwLCJ1cmwiOiJodHRwcyUzQSUyRiUyRnd3dy5jYXJzaWZ1Lm15JTJGIiwicGV0IjoxNjg4ODg5Mzg0MjgwLCJzZXQiOjE2ODg4ODkxMjgzODEsInBVcmwiOiJodHRwcyUzQSUyRiUyRnd3dy5jYXJzaWZ1Lm15JTJGYmlrZS1yZXZpZXdzIiwicFBldCI6MTY4ODg4OTM2NzcxNywicFR4IjoxNjg4ODg5MzY3NzE3fQ==',
}

headers = {
    'authority': 'www.carsifu.my',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': '_gid=GA1.2.1774589498.1688827808; _pcid=%7B%22browserId%22%3A%22lju4h0iqcndtfxp8%22%7D; cX_P=lju4h0iqcndtfxp8; _pctx=%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAEzIEYOAWAZgAYATAHZ%2BADl68xwwRwEBWQSAC%2BQA; __stgeo=IjAi; __stbpnenable=MQ==; __stdf=MA==; cX_G=cx%3A1i0t3fg74zq623k3k9afr0x7u3%3A2nq99z7wrkb6x; __stp=eyJ2aXNpdCI6InJldHVybmluZyIsInV1aWQiOiIwYzljYmIyZC02MTliLTQxMGItYTQzZS01YTg1N2EwMjhmODAifQ==; XSRF-TOKEN=eyJpdiI6ImZGQlwvVGVHSkc4UE9CMjk1ZGlpcEpBPT0iLCJ2YWx1ZSI6InhrbUlrM2FKZlpROWU5XC91bnFESnJ3WTV1NzBidDdHNXFtaExFUHBNOThXQVBmTVZUTXFTTmVJTzB3bHRra0crNnZOMEIzYUpUcHQ2Tk5ZNVpjNERPdz09IiwibWFjIjoiYzBlOGQ4ZGYxMjIzMzRhZjZmYjg3NjcxYTVjNDJmZDg0OGE1ZjljZWU5NmRkZmFlYTI4MmI3YTZmOTQ5NWQ1MiJ9; carsifu_session=eyJpdiI6IkZjUG1DYnQrTkcrS0k0c3ZkUjNxYUE9PSIsInZhbHVlIjoidmgzUm9xaGpqSFJQVkswb2J3SmtyQ3V4emlXbEI5REhuUlJ5R1drRmc3ZDk3UHQwOWVlNzB1TzhyVTYwZmJSamFIWGI2eWU3MGl3c3NCWlI0VnhYeHc9PSIsIm1hYyI6IjIzYWJmM2IyOTM4YzVjZjk4MmU1NzZjZGU5NTgxMjE0YzM4YmQwOGRjYmFiMDcyOTIzNDk4MDBhOTI5ZmM5NTIifQ%3D%3D; _ga_QF13J11YBW=GS1.1.1688889128.2.1.1688889384.35.0.0; _ga=GA1.1.2139008171.1688827808; __sts=eyJzaWQiOjE2ODg4ODkxMjgzODEsInR4IjoxNjg4ODg5Mzg0MjgwLCJ1cmwiOiJodHRwcyUzQSUyRiUyRnd3dy5jYXJzaWZ1Lm15JTJGIiwicGV0IjoxNjg4ODg5Mzg0MjgwLCJzZXQiOjE2ODg4ODkxMjgzODEsInBVcmwiOiJodHRwcyUzQSUyRiUyRnd3dy5jYXJzaWZ1Lm15JTJGYmlrZS1yZXZpZXdzIiwicFBldCI6MTY4ODg4OTM2NzcxNywicFR4IjoxNjg4ODg5MzY3NzE3fQ==',
    'pragma': 'no-cache',
    'referer': 'https://www.carsifu.my/bike-reviews',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

In [5]:
r = requests.get(urls[0], cookies = cookies, headers = headers)
soup = BeautifulSoup(r.content, "lxml")

In [11]:
soup.find('h2').text

'Alta bike hints at Harley-Davidson’s electric future'

In [9]:
urls[0]

'https://www.carsifu.my/bike-reviews/alta-bike-hints-at-harley-davidsons-electric-future'

In [15]:
soup.find('div', {'id': 'post-content'}).text

'\n                When Harley-Davidson Inc. announced earlier this year that it would make an electric motorcycle, many people in the independent motorcycle community lauded the move.\n\r\n“This will only increase the respect that the brand gets,” Alan Stulberg, founder of custom-build shop Revival Motorcycles, said at the time. “At least finally, they can say that Harley does low-tech heritage and it does cutting-edge technology.”\n\r\nBut the brand didn’t say it would make the e-bike from scratch. So it was interesting to note which company Harley announced in March would help usher it into the new millennium, as it were: Alta Motors, the California-based manufacturer founded in 2010.\n\n\n\r\nThis made sense. Harley has suffered for years from declining sales as its rider base enters retirement. Alta’s average customer age is younger, and the company undoubtedly faces the needs intrinsic to any startup-help with infrastructure and funding.\n\r\nThose who have followed closely may h

In [ ]:
for no, url in tqdm(enumerate(urls)):
    filename = os.path.join('page', f'{no}.json')
    if os.path.exists(filename):
        continue
    
    while True:
        try:
            r = requests.get(url, cookies = cookies, headers = headers)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")

    try:
        title = soup.find('h2').text
    except Exception as e:
        title = None

    try:
        body = soup.find('div', {'id': 'post-content'}).text
    except Exception as e:
        body = None

    data = {
        'url': url,
        'title': title,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

17797it [3:38:50,  2.43it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

22449it [4:36:30,  1.03it/s]